<a href="https://colab.research.google.com/github/YasinEnigma/mci-chatbot/blob/main/task-005/nemo_voice_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyTelegramBotAPI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 213 kB 16.8 MB/s 
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.7.1-py3-none-any.whl size=196988 sha256=9482bc9e077d58d3e626fd8a0f956b13841dd661964df374842ca0abf8019f8f
  Stored in directory: /root/.cache/pip/wheels/84/72/5f/8a7b3ca1ef1250002cf3bac48e0c72c5ee5c9596a5b1295dc1
Successfully built pyTelegramBotAPI


In [2]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
5. Restart the runtime (Runtime -> Restart Runtime) for any upgraded packages to take effect
"""
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg libsox-fmt-mp3
!pip install unidecode
!pip install matplotlib>=3.3.2

## Install NeMo
BRANCH = 'r1.11.0'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

## Grab the config we'll use in this example
!mkdir configs
!wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/config.yaml

"""
Remember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!
Alternatively, you can uncomment the exit() below to crash and restart the kernel, in the case
that you want to use the "Run All Cells" (or similar) option.
"""
# exit()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=9ffe3f95a408f45d6adf961fab6ab4f6da8d8efb7ac258b79e4a961597463eb6
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-4ubuntu0.18.04.2).
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libid3tag0 libmad0 libmagic-mgc libmagic1 libopencore-amrnb0
  libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base libsox3
Suggested packages:
  file libsox-fmt-all
The following NEW packages will be installed:
  

'\nRemember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!\nAlternatively, you can uncomment the exit() below to crash and restart the kernel, in the case\nthat you want to use the "Run All Cells" (or similar) option.\n'

In [3]:
#downloading pretrained models:
!git clone https://github.com/YasinEnigma/mci-chatbot

Cloning into 'mci-chatbot'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 78 (delta 22), reused 17 (delta 2), pack-reused 9
Unpacking objects: 100% (78/78), done.


In [4]:
import nemo
import nemo.collections.asr as nemo_asr
from nemo.collections.asr.metrics.wer import word_error_rate
from nemo.utils import logging, exp_manager
import wave
import json
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read 


import soundfile as sf
import json
import IPython
import librosa
import telebot
import requests
import warnings
warnings.filterwarnings('ignore')

[NeMo W 2022-11-03 06:11:28 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.


In [5]:
asr_model = nemo_asr.models.EncDecCTCModel.restore_from(restore_path='mci-chatbot/task-003/asr_model.h')

[NeMo W 2022-11-03 06:11:33 modelPT:143] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: manifests/fa/commonvoice_train_manifest.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - i
    - m
    - 'n'
    - p
    - r
    - s
    - t
    - 'y'
    - z
    - ا
    - ب
    - ت
    - ث
    - ج
    - ح
    - خ
    - د
    - ذ
    - ر
    - ز
    - س
    - ش
    - ص
    - ض
    - ط
    - ظ
    - ع
    - غ
    - ف
    - ق
    - ل
    - م
    - ن
    - ه
    - و
    - پ
    - چ
    - ژ
    - ک
    - گ
    - ی
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    num_workers: 8
    pin_memory: true
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    
[N

[NeMo I 2022-11-03 06:11:33 features:223] PADDING: 16


ERROR:numba.cuda.cudadrv.driver:Call to cuInit results in CUDA_ERROR_NO_DEVICE


[NeMo I 2022-11-03 06:11:34 save_restore_connector:243] Model EncDecCTCModel was successfully restored from /content/mci-chatbot/task-003/asr_model.h.


In [6]:
API_TOKEN = ''

In [9]:
bot = telebot.TeleBot(API_TOKEN)

# Handle '/start' and '/help'
@bot.message_handler(commands=['help', 'start'])
def send_welcome(message):
    bot.reply_to(message, """\
Hi there, I am VoiceBot.
I am here to recieve your voices and return transcript back to you. Just say anything nice :)) \
""")

# Handle all other messages with content_type 'text' (content_types defaults to ['text'])
@bot.message_handler(func=lambda message: True)
def echo_message(message):
    bot.reply_to(message, message.text)

@bot.message_handler(content_types=['voice'])
def read_message(message):
	file_info = bot.get_file(message.voice.file_id)
	file = requests.get('https://api.telegram.org/file/bot{0}/{1}'.format(API_TOKEN, file_info.file_path))
	with open('voice.ogg','wb') as f:
		f.write(file.content)
	x,_ = librosa.load('voice.ogg', sr=16000)
	sf.write('voice.wav', x, 16000)
	bot.send_message(message.chat.id, 'File is Recieved!')
 
	text = asr_model.transcribe(paths2audio_files=['voice.wav'], batch_size=1)[0]
	bot.reply_to(message, f'Text is: \n {text}')

In [10]:
if __name__ == '__main__':
    bot.polling(none_stop=True)


bot.infinity_polling()

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

2022-11-03 06:20:19,573 (__init__.py:972 MainThread) ERROR - TeleBot: "Break infinity polling"
ERROR:TeleBot:Break infinity polling
